In [1]:
import json
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType

In [2]:
# Criando SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

21/11/07 17:22:17 WARN Utils: Your hostname, alex-Inspiron-5566 resolves to a loopback address: 127.0.1.1; using 192.168.18.49 instead (on interface wlp2s0)
21/11/07 17:22:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/07 17:22:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
## Corrigir gambiarra depois

# # Separando os items do JSON
# for pais in lista_paises:
#     with open('archive/'+ pais +'_category_id.json') as categories:
#         obj = json.load(categories)
#         items = obj['items']

#         with open('archive/'+ pais +'_category_id.json', 'w') as f:
#             json.dump(items, f)

In [4]:
lista_paises = ['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']
base_name_csv = 'videos.csv'
base_name_json = '_category_id.json'
path = 'archive/'
videos = {}
categorias = {}
todos = None

In [5]:
colunas = ['video_id', 'trending_date', 'title', 'category_id', 'publish_time', 'views', 'likes', 'dislikes', \
           'comment_count']

## Lendo os arquivos de vídeos

In [6]:
# Comando de leitura

for pais in lista_paises:
    categorias[pais] = spark.read.format('org.apache.spark.sql.json').option("multiline","true") \
        .load(path+pais+base_name_json)\
        .select("id", f.col("snippet.title").alias('category_title'))
    
    videos[pais] = spark.read.format("csv").option("header", "true").load(path+pais+base_name_csv) \
        .select(colunas) \
        .join(categorias[pais], f.col('category_id') == f.col('id')) \
        .drop('category_id') \
        .drop('id')
    

In [7]:
# Comando de leitura
videos['US']['video_id', 'trending_date', 'views', 'category_title', 'publish_time'].show(truncate=False)

+-----------+-------------+-------+--------------------+------------------------+
|video_id   |trending_date|views  |category_title      |publish_time            |
+-----------+-------------+-------+--------------------+------------------------+
|2kyS6SvSYSE|17.14.11     |748374 |People & Blogs      |2017-11-13T17:13:01.000Z|
|1ZAPwfrtAFY|17.14.11     |2418783|Entertainment       |2017-11-13T07:30:00.000Z|
|5qpjK5DgCt4|17.14.11     |3191434|Comedy              |2017-11-12T19:05:24.000Z|
|puqaWrEC7tY|17.14.11     |343168 |Entertainment       |2017-11-13T11:00:04.000Z|
|d380meD0W0M|17.14.11     |2095731|Entertainment       |2017-11-12T18:01:41.000Z|
|gHZ1Qz0KiKM|17.14.11     |119180 |Science & Technology|2017-11-13T19:07:23.000Z|
|39idVpFF7NQ|17.14.11     |2103417|Entertainment       |2017-11-12T05:37:17.000Z|
|nc99ccSXST0|17.14.11     |817732 |Science & Technology|2017-11-12T21:50:37.000Z|
|jr9QtXwC9vc|17.14.11     |826059 |Film & Animation    |2017-11-13T14:00:23.000Z|
|TUmyygCMMGA|17.

## Resultados Leitura

In [8]:
## Mostra o schema do Dataframe
videos['US'].printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- category_title: string (nullable = true)



In [9]:
categorias['US'].printSchema()

root
 |-- id: string (nullable = true)
 |-- category_title: string (nullable = true)



In [10]:
categorias['US'].select('*').show()

+---+--------------------+
| id|      category_title|
+---+--------------------+
|  1|    Film & Animation|
|  2|    Autos & Vehicles|
| 10|               Music|
| 15|      Pets & Animals|
| 17|              Sports|
| 18|        Short Movies|
| 19|     Travel & Events|
| 20|              Gaming|
| 21|       Videoblogging|
| 22|      People & Blogs|
| 23|              Comedy|
| 24|       Entertainment|
| 25|     News & Politics|
| 26|       Howto & Style|
| 27|           Education|
| 28|Science & Technology|
| 29|Nonprofits & Acti...|
| 30|              Movies|
| 31|     Anime/Animation|
| 32|    Action/Adventure|
+---+--------------------+
only showing top 20 rows



### 3. Unindo Dataframes

In [11]:
schema = videos['US'].schema
df_videos = spark.createDataFrame([], schema)

In [12]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- category_title: string (nullable = true)



In [13]:
for pais in lista_paises:
    df_videos = df_videos.union(videos[pais])

In [14]:
df_videos = df_videos.withColumn('views', f.col('views').cast('int')) \
                     .withColumn('likes', f.col('likes').cast('int')) \
                     .withColumn('dislikes', f.col('dislikes').cast('int')) \
                     .withColumn('comment_count', f.col('comment_count').cast('int')) \
                     .withColumn('trending_date', f.to_timestamp(f.col('trending_date'), 'yy.dd.MM')) \
                     .withColumn('publish_time', f.regexp_replace('publish_time', 'T', ' ')) \
                     .withColumn('publish_time', f.regexp_replace('publish_time', 'Z', '')) \
                     .withColumn('publish_time', f.to_timestamp(f.col('publish_time'), 'yyyy-MM-dd HH:mm:ss.SSS'))  

### 4. Lidando com valores nulos

In [15]:
df_videos = df_videos.na.fill(value=0,subset=['comment_count', 'likes', 'dislikes'])

### 5. Criando coluna interations

In [16]:
df_videos = df_videos.withColumn('interation', f.col('likes')+f.col('dislikes')+f.col('comment_count'))

In [17]:
df_videos.select('video_id', 'comment_count', 'likes', 'dislikes', 'interation').show(truncate=False)

+-----------+-------------+-------+--------+----------+
|video_id   |comment_count|likes  |dislikes|interation|
+-----------+-------------+-------+--------+----------+
|n1WpP7iowLc|125882       |787425 |43420   |956727    |
|0dBIkQ4Mz1M|13030        |127794 |1688    |142512    |
|5qpjK5DgCt4|8181         |146035 |5339    |159555    |
|d380meD0W0M|17518        |132239 |1989    |151746    |
|2Vv-BfVoq4g|85067        |1634130|21082   |1740279   |
|0yIWz1XEeyc|12143        |103755 |4613    |120511    |
|_uM5kFfkhB8|26629        |187464 |9850    |223943    |
|2kyS6SvSYSE|15959        |57534  |2967    |76460     |
|JzCsM1vtn78|36391        |292837 |4123    |333351    |
|43sm-QwLcx4|1484         |4135   |976     |6595      |
|H1KBHFXm2Bg|28976        |263596 |8585    |301157    |
|U3xLOo-CNwo|3312         |28451  |2285    |34048     |
|FyZMnhUtLfE|186          |218    |30      |434       |
|7MxiQ4v0EnE|8063         |127477 |7134    |142674    |
|LUzsOyWp9lw|5335         |31810  |668     |3781

### 6. Criando coluna time_to_trends

In [18]:
df_videos = df_videos.withColumn('time_to_trends', f.col('trending_date').cast('long') - f.col('publish_time').cast('long'))

In [19]:
df_videos.select('video_id', 'trending_date', 'views', 'category_title', 'publish_time', 'time_to_trends').show(truncate=False)

+-----------+-------------------+--------+----------------+-------------------+--------------+
|video_id   |trending_date      |views   |category_title  |publish_time       |time_to_trends|
+-----------+-------------------+--------+----------------+-------------------+--------------+
|n1WpP7iowLc|2017-11-14 00:00:00|17158579|Music           |2017-11-10 17:00:03|284397        |
|0dBIkQ4Mz1M|2017-11-14 00:00:00|1014651 |Comedy          |2017-11-13 17:00:00|25200         |
|5qpjK5DgCt4|2017-11-14 00:00:00|3191434 |Comedy          |2017-11-12 19:05:24|104076        |
|d380meD0W0M|2017-11-14 00:00:00|2095828 |Entertainment   |2017-11-12 18:01:41|107899        |
|2Vv-BfVoq4g|2017-11-14 00:00:00|33523622|Music           |2017-11-09 11:04:14|392146        |
|0yIWz1XEeyc|2017-11-14 00:00:00|1309699 |News & Politics |2017-11-13 07:37:51|58929         |
|_uM5kFfkhB8|2017-11-14 00:00:00|2987945 |Comedy          |2017-11-12 23:52:13|86867         |
|2kyS6SvSYSE|2017-11-14 00:00:00|748374  |People &

In [20]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- category_title: string (nullable = true)
 |-- interation: integer (nullable = true)
 |-- time_to_trends: long (nullable = true)



### 7. Criando Dataframe de Categoria

In [21]:

categorias_text = df_videos.select('category_title').distinct().collect()
ids = [(idx+1, row.category_title) for idx, row in enumerate(categorias_text)]
ids

[(1, 'Shows'),
 (2, 'Education'),
 (3, 'Gaming'),
 (4, 'Entertainment'),
 (5, 'Travel & Events'),
 (6, 'Science & Technology'),
 (7, 'Sports'),
 (8, 'Howto & Style'),
 (9, 'Nonprofits & Activism'),
 (10, 'Film & Animation'),
 (11, 'People & Blogs'),
 (12, 'News & Politics'),
 (13, 'Pets & Animals'),
 (14, 'Movies'),
 (15, 'Autos & Vehicles'),
 (16, 'Music'),
 (17, 'Comedy'),
 (18, 'Trailers')]

In [22]:
df_categorias = spark.createDataFrame(ids).toDF('id', 'category_name')


In [23]:
df_categorias.show()

+---+--------------------+
| id|       category_name|
+---+--------------------+
|  1|               Shows|
|  2|           Education|
|  3|              Gaming|
|  4|       Entertainment|
|  5|     Travel & Events|
|  6|Science & Technology|
|  7|              Sports|
|  8|       Howto & Style|
|  9|Nonprofits & Acti...|
| 10|    Film & Animation|
| 11|      People & Blogs|
| 12|     News & Politics|
| 13|      Pets & Animals|
| 14|              Movies|
| 15|    Autos & Vehicles|
| 16|               Music|
| 17|              Comedy|
| 18|            Trailers|
+---+--------------------+



### 8. Atualizando ID das categorias no DataFrame de Videos

In [24]:
df_videos = df_videos.join(df_categorias, f.col('category_title') == f.col('category_name')) \
                    .withColumnRenamed('id', 'category_id') \
                    .drop('category_title')

In [25]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- interation: integer (nullable = true)
 |-- time_to_trends: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_name: string (nullable = true)



### 9. Criar coluna qtd_view no DataFrame de Categorias

In [26]:
df_categorias = df_categorias.join( \
                    df_videos.groupBy('category_id').agg(f.sum('views').alias('qtd_views')), \
                    f.col('id') == f.col('category_id')) \
                    .drop('category_id')
                    

In [27]:
df_categorias.show()

+---+--------------------+------------+
| id|       category_name|   qtd_views|
+---+--------------------+------------+
|  7|              Sports| 18972425164|
|  6|Science & Technology|  9194715151|
|  9|Nonprofits & Acti...|   168941392|
| 17|              Comedy| 22050866339|
|  5|     Travel & Events|   726674959|
|  1|               Shows|   444064556|
| 10|    Film & Animation| 27619339220|
|  3|              Gaming|  7730729502|
| 12|     News & Politics| 10422447730|
|  8|       Howto & Style|  9771031927|
| 11|      People & Blogs| 23600365409|
|  2|           Education|  2734841410|
|  4|       Entertainment|104517467253|
| 13|      Pets & Animals|  2008474231|
| 18|            Trailers|       55043|
| 14|              Movies|    70359777|
| 15|    Autos & Vehicles|  1661853766|
| 16|               Music|255967088943|
+---+--------------------+------------+

